### Imports

In [1]:
import gym
import random
import warnings
from time import sleep
warnings.filterwarnings('ignore')

### Creating the gym

In [2]:
env = gym.make('CartPole-v0')
states=env.observation_space.shape[0]
actions=env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
#Testing random episodes
episodes = 10

for episodes in range(1,episodes+1):
    state = env.reset()
    done  = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state , reward , done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episodes,score))

sleep(5)
env.close()

Episode:1 Score:19.0
Episode:2 Score:28.0
Episode:3 Score:44.0
Episode:4 Score:16.0
Episode:5 Score:14.0
Episode:6 Score:22.0
Episode:7 Score:20.0
Episode:8 Score:24.0
Episode:9 Score:22.0
Episode:10 Score:32.0


### Deep Reinforcement Learning NN

##### Keras Model

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(states,actions,activation_function='relu'):
    """
    build_model(states:int,actions:int)->Sequential:
    Builds a neural network model with a specified number of states and actions. The model consists of a flatten layer, two dense layers with 24 units each and ReLU activation, and a final dense layer with linear activation.
    Parameters:
    states (int): number of states for the model
    actions (int): number of actions for the model
    Returns:
    model (Sequential): the neural network model
    """
    model=Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(units=24,activation=activation_function))
    model.add(Dense(units=24,activation=activation_function))
    model.add(Dense(units=actions,activation='linear'))
    return model

##### Keras-reinforcement learning Model

In [6]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
def build_agent(model,actions):
    """
    build_agent(model:Sequential,actions:int)->DQNAgent:
    Builds a DQN agent with a specified neural network model and number of actions.
    The agent is built using a BoltzmannQPolicy, SequentialMemory with a limit of 50000 and window_length of 1, and a target_model_update of 1e-2.
    Parameters:
    model (Sequential): neural network model
    actions (int): number of actions for the agent
    Returns:
    dqn (DQNAgent): the DQN agent
    """
    policy  = BoltzmannQPolicy()
    memory  = SequentialMemory(limit=50000,window_length=1)
    dqn     = DQNAgent(model=model,memory=memory,policy = policy,
                       nb_actions = actions ,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

### Building the keras model and training the agent

In [8]:
model = build_model(states,actions,activation_function='LeakyReLU')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [9]:
env = gym.make('CartPole-v0')
states=env.observation_space.shape[0]
actions=env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [10]:
%%time
agent = build_agent(model,actions)
agent.compile(Adam(learning_rate=1e-3),metrics=['mae'])
agent.fit(env,nb_steps=50000,visualize=False,verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 189s 19ms/step - reward: 1.0000
119 episodes - episode_reward: 84.008 [9.000, 200.000] - loss: 3.182 - mae: 18.130 - mean_q: 36.644

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 182s 18ms/step - reward: 1.0000
50 episodes - episode_reward: 199.420 [185.000, 200.000] - loss: 7.325 - mae: 40.320 - mean_q: 81.633

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 191s 19ms/step - reward: 1.0000
50 episodes - episode_reward: 199.620 [189.000, 200.000] - loss: 7.676 - mae: 44.093 - mean_q: 88.974

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 179s 18ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 8.843 - mae: 44.524 - mean_q: 89.647

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 152s 15ms/step - reward: 1.

### Testing the agent

In [11]:
scores = agent.test(env,nb_episodes  =100  , visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [12]:
_ = agent.test(env,nb_episodes =10  , visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200


### Saving the agent

In [13]:
agent.save_weights('agent_weights_leaky_RElu.h5f',overwrite=True)

In [14]:
del model
del agent
del env

### Reloading the agent

In [17]:
env = gym.make('CartPole-v0')
states=env.observation_space.shape[0]
actions=env.action_space.n
model = build_model(states,actions)
agent = build_agent(model,actions)
agent.compile(Adam(learning_rate=1e-3),metrics=['mae'])
agent.load_weights('agent_weights.h5f')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Exception ignored in: <function Viewer.__del__ at 0x0000023F9C769160>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 143, in __del__
    self.close()
  File "C:\ProgramData\Anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 62, in close
    self.window.close()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 336, in close
    super(Win32Window, self).close()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyglet\window\__init__.py", line 857, in close
    app.windows.remove(self)
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000024167FB5D60; to 'Win32Window' at 0x0000024165A0CB50>


In [ ]:
_ = agent.test(env,nb_episodes  =20  , visualize=True)

Testing for 20 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
